In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import re

import read_nml as rnl

from Utils import numerical_utils as nuti
from Utils import constants as Co

print( Co.cpair())

cpair = Co.cpair()
grav= Co.grav()

1004.6480897776704


In [2]:

#################
# set case name
#################
#case='camsnap-Baseline01'
case='xy-Baseline-v3.0'
case='xy-testfrontg'
#case='xy-vortsrc-03'

year,month=2004,7

f=f'/glade/derecho/scratch/juliob/archive/GW_UnitTest/{case}/{case}.h.{year:04}-{month:02}-*.nc'     #        camsnap-yaaaa.h.2004-06-15-21600.nc'
title=f'{case}: {year:04}-{month:02}'     #        camsnap-yaaaa.h.2004-06-15-21600.nc'

Xgw=xr.open_mfdataset( f   ,data_vars='different', coords='different' )

ncdata=Xgw.ncdata
X=xr.open_dataset( ncdata )

print(ncdata)



ftopo='/glade/work/juliob/Topo/NCARTopoJTB/cases/fv1x1_Sco100_GrnlAnt/output/fv1x1_gmted2010_modis_bedmachine_nc3000_Laplace0100_noleak_greenlndantarcsgh30fac2.50_20251009.nc'
Topo=xr.open_dataset( ftopo )

/glade/derecho/scratch/juliob/archive//c153_topfix_ne240pg3_FMTHIST_xic_x02/atm/fv1x1/c153_topfix_ne240pg3_FMTHIST_xic_x02.cam.h1i.2004-07-02-00000.nc


In [5]:
Xgw

<xarray.Dataset> Size: 395MB
Dimensions:          (ny: 192, nx: 288, ncol: 55296, time: 1, level: 93,
                      ilevel: 94)
Coordinates:
  * time             (time) float64 8B 2.004e+07
Dimensions without coordinates: ny, nx, ncol, level, ilevel
Data variables: (12/31)
    lat_R            (ny) float64 2kB dask.array<chunksize=(192,), meta=np.ndarray>
    lon_R            (nx) float64 2kB dask.array<chunksize=(288,), meta=np.ndarray>
    lat              (ncol) float32 221kB dask.array<chunksize=(55296,), meta=np.ndarray>
    lon              (ncol) float32 221kB dask.array<chunksize=(55296,), meta=np.ndarray>
    SGH              (ncol) float32 221kB dask.array<chunksize=(55296,), meta=np.ndarray>
    ZM               (time, level, ncol) float32 21MB dask.array<chunksize=(1, 47, 27648), meta=np.ndarray>
    ...               ...
    VTRDG            (time, level, ncol) float32 21MB dask.array<chunksize=(1, 47, 27648), meta=np.ndarray>
    TAU_SRC_MOVMVTN  (time, ilevel, ncol) float32 21MB dask.array<chunksize=(1, 47, 27648), meta=np.ndarray>
    UBM              (time, level, ncol) float32 21MB dask.array<chunksize=(1, 47, 27648), meta=np.ndarray>
    ZETA             (time, level, ncol) float32 21MB dask.array<chunksize=(1, 47, 27648), meta=np.ndarray>
    TAU_MOVMTN       (time, ilevel, ncol) float32 21MB dask.array<chunksize=(1, 47, 27648), meta=np.ndarray>
    TAU_DIAG_MOVMTN  (time, ilevel, ncol) float32 21MB dask.array<chunksize=(1, 47, 27648), meta=np.ndarray>
Attributes:
    ncdata:            /glade/derecho/scratch/juliob/archive//c153_topfix_ne2...
    calculation_type:  xy
    Conventions:       CF-1.8

In [ ]:
print(Xgw.time.values)

In [ ]:
#read GW.dat file

%run -i val_camsnap_block.py


In [ ]:
ncdata=Xgw.ncdata

In [ ]:
print( ncdata )

In [ ]:


#s = "/glade/derecho/scratch/juliob/archive//c153_topfix_ne240pg3_FMTHIST_xic_x02/atm/fv1x1/c153_topfix_ne240pg3_FMTHIST_xic_x02.cam.h1i.2004-07-01-00000.nc"

# Replace date tag of the form YYYY-MM-DD-##### with '*'
#ncdata_star = re.sub(r'\d{4}-\d{2}-\d{2}-\d{5}', '*', ncdata)
ncdata_star = re.sub(r'\d{2}-\d{5}', '*', ncdata)

print(ncdata_star)


In [ ]:
X=xr.open_mfdataset( ncdata_star   ,data_vars='different', coords='different' )

In [ ]:
X

In [ ]:
Xgw

In [ ]:
%%time

tau_rdg=Xgw.TAU_RDG.values
tau_mmt=Xgw.TAU_MOVMTN.values

upwp = X.upwp.values
vpwp = X.vpwp.values


In [ ]:
epwp = np.sqrt( upwp**2 + vpwp**2 )


In [ ]:

print( Xgw.sizes )#[('ny','nx')])
nt,nz,ny,nx = Xgw.sizes['time'],  Xgw.sizes['level'],  Xgw.sizes['ny'],  Xgw.sizes['nx']

In [ ]:
%%time
tau_rdg = tau_rdg.reshape((nt,nz+1,ny,nx) )
tau_mmt = tau_mmt.reshape((nt,nz+1,ny,nx) )

tau_tot = tau_rdg + tau_mmt

In [ ]:
tau_rdg_tv = np.average( tau_rdg, axis=0)
tau_mmt_tv = np.average( tau_mmt, axis=0)
tau_tot_tv = np.average( tau_tot, axis=0)

upwp_tv = np.average(upwp,axis=0)
epwp_tv = np.average(epwp,axis=0)


In [ ]:
from scipy.ndimage import uniform_filter
from scipy.ndimage import gaussian_filter
import matplotlib.colors as mcolors

# A: 2D array (ny, nx)
#A_smooth = uniform_filter(A, size=3, mode="nearest")   # 3x3
# A_smooth = uniform_filter(A, size=5, mode="nearest") # 5x5


In [ ]:
htopo=Topo.PHIS.values/grav

In [ ]:
zlev=-7_000.*np.log( X.lev.values /1_000.)
lat=X.lat.values
lon=X.lon.values

In [ ]:
z=5

title=f'{case}: {year:04}-{month:02}, Z={zlev[z]/1000.:.0f} km '     #        camsnap-yaaaa.h.2004-06-15-21600.nc'

tau_tot_smooth = uniform_filter(   tau_tot_tv[z,:,:]  , size=7, mode="wrap")   # 3x3

trunc_k, sigma=7, 1.5
tau_mmt_smooth = gaussian_filter(  tau_mmt_tv[z,:,:] , sigma=sigma,
                           truncate=((trunc_k-1)/2)/sigma, mode="nearest")
tau_tot_smooth = gaussian_filter(  tau_tot_tv[z,:,:] , sigma=sigma,
                           truncate=((trunc_k-1)/2)/sigma, mode="nearest")


#flev=[0.0001,0.0002,0.0005,  0.001,0.002,0.005,  0.01,0.02,0.05,  0.1,0.2,0.5,   1.0  ]
flev=[0.0001,0.0002,0.0005,  0.001,0.0015,0.002,0.003,0.004,0.005,0.006,0.008,    0.01,0.015,0.02,0.03,0.04,0.05,0.06,  0.1,0.2,0.5,   1.0  ]
flev=0.01*np.asarray( flev )
print(flev)
cmapN='gist_ncar'

cmap = plt.cm.bwr  #gist_ncar  # Or any other colormap
cmap = plt.cm.gist_ncar # .plasma #gist_ncar  #gist_ncar  # Or any other colormap
norm = mcolors.BoundaryNorm(boundaries=flev, ncolors=cmap.N, clip=False)

topocolor='white' #'black'


fig,axs=plt.subplots( 1, 3, figsize=(30,6) )
ax=axs[0]
c = ax.contourf( lon, lat, tau_tot_tv[z,:,:],levels=flev, cmap=cmap , norm=norm, extend='both'  )
to = ax.contour( lon, lat, htopo, levels=[1,100] ,colors=topocolor)
ax.set_title( " Param GW "+ r"$\tau$")
plt.colorbar( c )

ax=axs[1]
c = ax.contourf( lon, lat, tau_tot_smooth,levels=flev, cmap=cmap , norm=norm, extend='both'  )
to = ax.contour( lon, lat, htopo, levels=[1,100] ,colors=topocolor)
ax.set_title( " Smoothed param GW "+ r"$\tau$")
plt.colorbar( c )


ax=axs[2]
c = ax.contourf( lon, lat, epwp_tv[z,:,:],levels=flev, cmap=cmap , norm=norm, extend='both'  )
to = ax.contour( lon, lat, htopo, levels=[1,100] ,colors=topocolor)
ax.set_title( f'Resolved GW {r"$\tau$"} in 14km sim.')
plt.colorbar( c )

plt.suptitle( title , fontsize=24)

print(X.lev[z].values)
print(zlev[z])


In [ ]:
print(zlev[z])
print(X.lev[z].values)

In [ ]:
plt.plot( axv[:,30:50,0:100].flatten() )